In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/Users/apple/Desktop/Kaggle-Competition-Titanic/models')
from collections import Counter
import kmodes

In [51]:
df = pd.read_csv('train_processed.csv', usecols=range(1,10), dtype=str)
print(df.shape)
df.drop(['Survived'], axis=1).head()

(891, 9)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,3,0,1,1,0,0,2,1
1,1,1,2,1,0,3,0,3
2,3,1,1,0,0,1,2,2
3,1,1,2,1,0,3,2,3
4,3,0,2,0,0,1,2,1


In [43]:
test = pd.read_csv('test_processed.csv', usecols=range(1,9),dtype=str)
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,3,0,2,0,0,0,1,1
1,3,1,2,1,0,0,2,3
2,2,0,3,0,0,1,1,1
3,3,0,1,0,0,1,2,1
4,3,1,1,1,1,1,2,3


In [56]:
data = 'train_processed.csv'
# data = kmodes.get_data(data)
# X = data[:800]
# Y = data[800:]
df = pd.read_csv('train_processed.csv', usecols=range(1,9), dtype=str)
actual = df[800:]['Survived'].values.flatten()
test_data = df[800:].drop(['Survived'], axis=1).values.tolist()
# test_data[:5]
test_data[:5]
# actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str).values.flatten()

[['2', '0', '2', '0', '0', '1', '2'],
 ['2', '1', '1', '1', '1', '2', '2'],
 ['1', '0', '0', '1', '1', '3', '2'],
 ['3', '0', '0', '0', '1', '1', '0'],
 ['3', '0', '1', '0', '0', '0', '2']]

## KNN

In [100]:
# KNN K=22
data = 'train_processed.csv'
X = kmodes.get_data(data)
predictions = []
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9),dtype=str).values.tolist()
K = 22
prediction = []

def getkey(item):
    return item[0]

def get_label(item):
    lst = [each[1] for each in item]
#     print(lst)
    return Counter(lst).most_common(1)[0][0]

for i,x in enumerate(test_data):
    closest = []
    for pt in X:
        d = kmodes.get_distance(pt[:-1],x)
        l = pt[-1]
        closest.append((d,l))
    closest.sort(key=getkey)
    closest = closest[:K]
    label = get_label(closest)
    prediction.append(label)
#     print(label)
    
test_ID = pd.read_csv('gender_submission.csv')
submission = pd.DataFrame({
        "PassengerId": test_ID["PassengerId"],
        "Survived": prediction
    })
print(submission.head())
submission.to_csv('titanic.csv', index=False)
print('The best accuracy on Kaggle is 0.78469')

   PassengerId Survived
0          892        0
1          893        1
2          894        0
3          895        0
4          896        1
The best accuracy on Kaggle is 0.78469


## kmodes Model with KNN

In [67]:
# kmodes Model with KNN
# from sklearn.model_selection import train_test_split
data = 'train_processed.csv'
X = kmodes.get_data(data)
# print(X[:4])
scores = []
predictions = []
for i in range(5):
    num_instances = len(X)
    # store the best records in 5 iterations 
    best_score = 0
    best_centroids = []
    best_clusters =[]
    best_iteratoin = 0
    pred = []
    for i in range(10):
        centroids, clusters , iteration= kmodes.find_centers(X, 2)
        purity = kmodes.get_purity(clusters, centroids, num_instances)
        if purity > best_score:
            best_centroids = centroids
            best_clusters = clusters
            best_score = purity
            best_iteratoin = iteration
    actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str).values.flatten()
    test_data = pd.read_csv('test_processed.csv', usecols=range(1,9),dtype=str).values.tolist()
    for idx, c in enumerate(best_centroids):
        labels = np.array(best_clusters[idx])[:,-1]
        label = Counter(labels).most_common(1)[0][0]
        c[-1] = label
    count = 0
    for i, x in enumerate(test_data):
        distance = [kmodes.get_distance(x, c[:-1]) for c in best_centroids]
        idx = np.argsort(distance)[0]
        label = best_centroids[idx][-1]
    #     print(actual[i])
    #     print('%s vs %s' % (label, actual[i]))
        pred.append(label)
        if str(label) == str(actual[i]):
            count += 1
    predictions.append(pred)
    acc_kmode_knn = round(float(count)/len(actual)*100, 2)
    scores.append(acc_kmode_knn)
idx = np.argsort(scores)[0]
prediction = predictions[idx]
test_ID = pd.read_csv('gender_submission.csv')
submission = pd.DataFrame({
        "PassengerId": test_ID["PassengerId"],
        "Survived": prediction
    })
print(submission.head())
submission.to_csv('titanic.csv', index=False)
print('The best accuracy on Kaggle is 0.73206')

   PassengerId Survived
0          892        0
1          893        1
2          894        0
3          895        0
4          896        1
The best accuracy on Kaggle is 0.73206


## Decision Tree using Gain Ratio

In [68]:
# Decision Tree using Gain Ratio
import decisionTree_GR
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
# print(train_data.head())
test_data = pd.concat([actual, test_data], axis=1)
# print(test_data.head())
actual = actual.values.flatten()
rows = train_data.values.tolist()
# print(rows[:2])
test_rows = test_data.values.tolist()
root = decisionTree_GR.buildtree(rows)
decisionTree_GR.prune(root, 0.49)
predictions = []
# print(actual)
for row in test_rows:
    predicted = decisionTree_GR.classify(row, root).most_common(1)[0][0]
    predictions.append(predicted)
# print(predictions)
test_ID = pd.read_csv('gender_submission.csv')
submission = pd.DataFrame({
        "PassengerId": test_ID["PassengerId"],
        "Survived": predictions
    })
print(submission.head())
submission.to_csv('titanic.csv', index=False)
print('The best accuracy for C4.5 decision tree is 0.78469')

   PassengerId Survived
0          892        0
1          893        1
2          894        0
3          895        0
4          896        0
The best accuracy for C4.5 decision tree is 0.78469


## Decision Tree using Information Gain

In [73]:
# Decision Tree using Information Gain
import decisionTree_IG
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
# print(train_data.head())
test_data = pd.concat([actual, test_data], axis=1)
# print(test_data.head())
actual = actual.values.flatten()
rows = train_data.values.tolist()
# print(rows[:2])
test_rows = test_data.values.tolist()
root = decisionTree_IG.buildtree(rows)
decisionTree_IG.prune(root, 0.46)
predictions = []
# print(actual)
for row in test_rows:
    predicted = decisionTree_IG.classify(row, root).most_common(1)[0][0]
    predictions.append(predicted)
# print(predictions)
test_ID = pd.read_csv('gender_submission.csv')
submission = pd.DataFrame({
        "PassengerId": test_ID["PassengerId"],
        "Survived": predictions
    })
print(submission.head())
submission.to_csv('titanic.csv', index=False)
print('The best accuracy for IG decision tree is 0.77033')

   PassengerId Survived
0          892        0
1          893        0
2          894        0
3          895        0
4          896        0
The best accuracy for IG decision tree is 0.77033


## Naive Bayers 

In [77]:
# Naive Bayers 
import NaiveBayers
from collections import defaultdict
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
test_data = pd.concat([actual, test_data], axis=1)
actual = actual.values.flatten()
rows = train_data.values.tolist()
test_rows = test_data.values.tolist()
classes = defaultdict(int)
# I create one more attribute that is unnecessary because I want to index to correspond to attributes
# attributes: list of dicts: key is 'classValue + attributeValue', value is its corresponding count
attributes = [defaultdict(int) for i in range(len(rows[0]))]
NaiveBayers.prepare(classes, attributes, rows)
predictions = NaiveBayers.test(classes, attributes, test_rows, actual)
test_ID = pd.read_csv('gender_submission.csv')
submission = pd.DataFrame({
        "PassengerId": test_ID["PassengerId"],
        "Survived": predictions
    })
print(submission.head())
submission.to_csv('titanic.csv', index=False)
print('The accuracy of Naive Bayers is 0.74641')       

   PassengerId Survived
0          892        0
1          893        1
2          894        0
3          895        0
4          896        1
The accuracy of Naive Bayers is 0.74641


## Binary-perceptron

In [80]:
# Binary-perceptron
import perceptron
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
train_X = train_data.drop(['Survived'], axis=1)
train_Y = train_data['Survived']
test_X = test_data
test_Y = actual
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc_train_X = enc.fit_transform(train_X)
enc_test_X = enc.fit_transform(test_X)
enc_train_X = pd.DataFrame(enc_train_X.toarray())
enc_test_X = pd.DataFrame(enc_test_X.toarray())
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler(feature_range=(-1,1))
enc_train_X = min_max_scaler.fit_transform(enc_train_X)
enc_test_X = min_max_scaler.fit_transform(enc_test_X)
enc_train_X = pd.DataFrame(enc_train_X)
enc_test_X = pd.DataFrame(enc_test_X)
lr_rates = [0.1,0.11,0.12,0.13,0.14]
# learning_rate =0.13 is the best
n_epochs = [15,20,22, 24, 25, 26, 28, 30]
# n_epoch is the best at 26
prediction = []
for n_epoch in n_epochs:
    for i in range(5):
        train_data = pd.concat([enc_train_X, train_Y], axis=1)
        test_X = enc_test_X
        actual = test_Y.values.astype('float')
        learning_rate = 0.13
        n_epoch = n_epoch
        prediction = perceptron.evaluate(train_data, test_X, actual, learning_rate, n_epoch)
test_ID = pd.read_csv('gender_submission.csv')
submission = pd.DataFrame({
        "PassengerId": test_ID["PassengerId"],
        "Survived": predictions
    })
print(submission.head())
submission.to_csv('titanic.csv', index=False)
print('The accuracy for %d GD is 0.74641')

   PassengerId Survived
0          892        0
1          893        1
2          894        0
3          895        0
4          896        1
The accuracy for %d GD is 0.74641


In [101]:
models = pd.DataFrame({
    'Model': ['KNN',' kmodes with KNN', 'Multinomial Naive Bayers', 
              'Decision Tree Gain Ratio', 'Decision Tree Information Gain', 'Perceptron'],
    'Score': ['0.78469','0.73206', '0.74641', '0.78469', '0.77033', '0.74641']})
models.sort_values(by='Score', ascending=False)

,Model,Score
0,KNN,0.78469
3,Decision Tree Gain Ratio,0.78469
4,Decision Tree Information Gain,0.77033
2,Multinomial Naive Bayers,0.74641
5,Perceptron,0.74641
1,kmodes with KNN,0.73206
